# **Dask: De Conceptos a Aplicaciones Prácticas**

## Objetivos del Curso
- Comprender qué es Dask y por qué es fundamental para big data
- Dominar los conceptos de computación distribuida y paralelización
- Procesar datasets grandes en máquina local
- Aplicar Dask en casos prácticos reales
- Resolver problemas comunes y errores frecuentes

## Estructura del Curso
1. Introducción y Fundamentos (15 min) - Qué es Dask?
2. Conceptos Técnicos (20 min) - Lazy Evaluation y Particiones
3. Comparación Práctica (15 min) - Pandas vs Dask
4. Caso Práctico (20 min) - Dataset de Vuelos Real
5. Optimización Avanzada (10 min) - Mejores Prácticas


## 1. INTRODUCCIÓN: QUÉ ES DASK?

Dask es una biblioteca paralela y distribuida en Python que permite escalar cálculos comunes a conjuntos de datos más grandes de los que pueden manejar pandas o numpy directamente.

### Qué problemas resuelve Dask?

- Memoria limitada: Procesar datos más grandes que la RAM disponible
- CPU subutilizada: Aprovechar todos los núcleos del procesador
- Escalabilidad: Desde laptop personal hasta clusters distribuidos
- API familiar: Sintaxis similar a pandas sin curva de aprendizaje pronunciada

### Ventajas principales:
- Sintaxis similar a pandas y numpy
- Manejo eficiente de datos que no entran en memoria
- Paralelismo automático y distribución inteligente
- Integración con múltiples fuentes de datos
- Ecosistema completo (DataFrame, Array, ML, Delayed)

### Instalación

Si aún no tienes instalado dask, ejecuta:


In [ ]:
# ! pip install dask

También puedes instalarlo junto con otras dependencias útiles:

In [ ]:
# ! pip install dask[dataframe]

### Comparación: Dask vs Pandas

| Característica | Pandas | Dask |
|----------------|--------|------|
| Datos en RAM | Sí (todo en memoria) | No (lazy evaluation) |
| Paralelización | No (procesamiento secuencial) | Sí (múltiples núcleos) |
| Tamaño máximo | Limitado por RAM | Sí (terabytes) |
| API familiar | - | Sí (muy similar) |
| Velocidad | Rápido para datasets pequeños | Más lento por operación, pero escalable |
| Memoria | Carga todo inmediatamente | Carga bajo demanda |


## 2. IMPORTACIÓN Y CONFIGURACIÓN

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import dask.dataframe as dd
import dask as dk  # Dask principal para obtener versión
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

print(f"Versión de pandas: {pd.__version__}")
print(f"Versión de dask: {dk.__version__}")
print("Imports completados exitosamente")

## 3. CONCEPTOS FUNDAMENTALES

### A. Lazy Evaluation (Evaluación Perezosa)

Concepto clave: Dask no ejecuta operaciones inmediatamente, las construye en un grafo de tareas que se ejecuta solo cuando:
- Llamamos .compute()
- Guardamos a disco
- Mostramos resultados (trigger implícito)

Esta estrategia permite:
- Inicialización rápida (no carga datos inmediatamente)
- Optimización global del plan de ejecución
- Planificación inteligente de operaciones

In [ ]:
# Demostración de Lazy Evaluation
print("Demostrando Lazy Evaluation...")
start_time = time.time()

# Crear un DataFrame de Dask (no se carga en memoria aún)
n_partitions = 4
df_demo = dd.from_pandas(
    pd.DataFrame({
        'id': range(100000),
        'valor': np.random.randn(100000),
        'categoria': np.random.choice(['A', 'B', 'C'], 100000),
        'fecha': pd.date_range('2023-01-01', periods=100000, freq='1min')
    }),
    npartitions=n_partitions
)

creation_time = time.time() - start_time
print(f"DataFrame creado en {creation_time:.4f} segundos")
print(f"Tipo: {type(df_demo)}")
print(f"Particiones: {df_demo.npartitions}")

print("\nPlan de ejecución (lazy - SIN ejecutar):")
print(df_demo)  # Solo muestra el plan, no ejecuta

In [ ]:
# Trigger la ejecución con .compute()
print("Ejecutando .compute()...")
start_time = time.time()

result = df_demo.compute()

execution_time = time.time() - start_time
print(f"Ejecutado en {execution_time:.2f} segundos")
print(f"Forma del resultado: {result.shape}")
print(f"Tipo: {type(result)}")

print("\nPrimeras 5 filas:")
result.head()

## 4. CASO PRÁCTICO: DATASET DE VUELOS

### Análisis de Datos de Aviación con Dask

Usaremos un dataset real de vuelos para demostrar el poder de Dask:

In [ ]:
# URLs de datasets de vuelos (de diferentes tamaños)
print("CONFIGURACIÓN DE DATOS DE VUELOS")
print("=" * 40)

big_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008.zip"
small_csv="https://github.com/ricardoahumada/Python_for_Data_Science/raw/refs/heads/master/data/2008_small.zip"
very_small_csv = 'https://github.com/ricardoahumada/data-for-auditors/raw/refs/heads/main/4.%20An%C3%A1lisis%20Masivo%20de%20Datos/Optimizacion/data/2008_very_small.csv'

print("Dataset disponible: vuelos pequeños (~7MB) para demo")
print("\nPara archivos grandes, Dask necesita dependencias adicionales:")
print("# !pip install requests aiohttp")

In [ ]:
# Cargar dataset con Dask (dataset muy pequeño para demo)
print("Cargando dataset con Dask...")
start_time = time.time()

try:
    df_vuelos = dd.read_csv(small_csv, dtype={'CancellationCode': 'object'})
    load_time = time.time() - start_time
    
    print(f"Dataset cargado en {load_time:.2f} segundos")
    print(f"Tipo: {type(df_vuelos)}")
    print(f"Particiones: {df_vuelos.npartitions}")
    
    # Ver estructura sin ejecutar completamente
    print(f"\nPlan de ejecución:")
    print(df_vuelos)
    
except Exception as e:
    print(f"Error cargando dataset: {e}")
    print("Usando dataset sintético alternativo...")
    
    # Dataset sintético alternativo
    df_vuelos = dd.from_pandas(
        pd.DataFrame({
            'Year': np.random.randint(2008, 2009, 10000),
            'Month': np.random.randint(1, 13, 10000),
            'DayofMonth': np.random.randint(1, 32, 10000),
            'DayOfWeek': np.random.randint(1, 8, 10000),
            'DepDelay': np.random.randint(-30, 300, 10000),
            'ArrDelay': np.random.randint(-30, 300, 10000),
            'Origin': np.random.choice(['JFK', 'LAX', 'ORD', 'ATL', 'DFW'], 10000),
            'Dest': np.random.choice(['JFK', 'LAX', 'ORD', 'ATL', 'DFW'], 10000),
            'Distance': np.random.randint(100, 3000, 10000)
        }),
        npartitions=4
    )
    print("Dataset sintético de vuelos creado")

In [ ]:
# Verificar estructura del DataFrame
print("ESTRUCTURA DEL DATASET DE VUELOS")
print("=" * 40)

# Mostrar las primeras filas (ejecuta el plan)
print("\nPrimeras 5 filas:")
df_vuelos.head()

In [ ]:
# Información de columnas
print("\nColumnas disponibles:")
print(df_vuelos.columns.tolist())

print("\nTipos de datos:")
print(df_vuelos.dtypes)

### Análisis de Vuelos con Dask

In [ ]:
# Análisis de retrasos de vuelos
print("ANÁLISIS DE VUELOS CON DASK")
print("=" * 40)

# 1. Filtrar vuelos con retraso
print("\n1. Vuelos con retraso de salida > 3 minutos:")
start = time.time()
vuelos_retrasados = df_vuelos[df_vuelos['DepDelay'] > 3]
resultado = vuelos_retrasados.compute()
print(f"Tiempo: {time.time() - start:.3f} segundos")
print(f"Vuelos con retraso: {len(resultado):,} de {len(df_vuelos.compute()):,} total")
print(f"Porcentaje: {(len(resultado)/len(df_vuelos.compute()))*100:.1f}%")

# 2. Seleccionar columnas específicas
print("\n2. Seleccionar columnas de fecha:")
fecha_cols = df_vuelos[['Year', 'Month', 'DayofMonth', 'DayOfWeek']]
print("Columnas seleccionadas: Year, Month, DayofMonth, DayOfWeek")
fecha_cols.head()

In [ ]:
# 3. Calcular promedios de retraso
print("\n3. Análisis de retrasos:")
start = time.time()

promedio_dep_delay = df_vuelos['DepDelay'].mean().compute()
promedio_arr_delay = df_vuelos['ArrDelay'].mean().compute()

print(f"Tiempo: {time.time() - start:.3f} segundos")
print(f"Retraso promedio salida: {promedio_dep_delay:.2f} minutos")
print(f"Retraso promedio llegada: {promedio_arr_delay:.2f} minutos")

# 4. Agrupamiento por aeropuerto de origen
print("\n4. Retrasos por aeropuerto de origen:")
start = time.time()
retrasos_origen = df_vuelos.groupby('Origin')['DepDelay'].mean().compute()
print(f"Tiempo: {time.time() - start:.3f} segundos")
print("\nRetrasos promedio por aeropuerto:")
for aeropuerto, retraso in retrasos_origen.sort_values(ascending=False).items():
    print(f"   {aeropuerto}: {retraso:.2f} minutos")

## 5. COMPARACIÓN PRÁCTICA: PANDAS VS DASK

### Performance en Dataset Real

Vamos a comparar el rendimiento en el mismo dataset:

In [ ]:
# Comparación directa Pandas vs Dask
print("COMPARACIÓN DE PERFORMANCE: PANDAS VS DASK")
print("=" * 55)

# Convertir a pandas para comparación
print("Convirtiendo a pandas para comparación...")
df_pandas = df_vuelos.compute()
print(f"Dataset pandas: {df_pandas.shape}")

print("\nComparación de operaciones básicas:")

# Operación 1: Filtrado
print("\n1. FILTRADO: Vuelos con DepDelay > 15")

# Pandas
start = time.time()
pandas_filtro = df_pandas[df_pandas['DepDelay'] > 15]
pandas_time = time.time() - start
pandas_result_count = len(pandas_filtro)

# Dask
start = time.time()
dask_filtro = df_vuelos[df_vuelos['DepDelay'] > 15].compute()
dask_time = time.time() - start
dask_result_count = len(dask_filtro)

print(f"   Pandas: {pandas_time:.4f} segundos - {pandas_result_count:,} resultados")
print(f"   Dask:   {dask_time:.4f} segundos - {dask_result_count:,} resultados")
print(f"   Ganador: {'Dask' if dask_time < pandas_time else 'Pandas'}")

# Operación 2: GroupBy
print("\n2. GROUPBY: Retrasos por aeropuerto")

# Pandas
start = time.time()
pandas_groupby = df_pandas.groupby('Origin')['DepDelay'].mean()
pandas_time = time.time() - start

# Dask
start = time.time()
dask_groupby = df_vuelos.groupby('Origin')['DepDelay'].mean().compute()
dask_time = time.time() - start

print(f"   Pandas: {pandas_time:.4f} segundos")
print(f"   Dask:   {dask_time:.4f} segundos")
print(f"   Ganador: {'Dask' if dask_time < pandas_time else 'Pandas'}")

## 6. OPTIMIZACIÓN AVANZADA

### Técnicas para Maximizar el Rendimiento de Dask

In [ ]:
# Optimización 1: Tipos de datos eficientes
print("OPTIMIZACIÓN: TIPOS DE DATOS")
print("=" * 35)

print("\nTipos de datos originales:")
print(df_vuelos.dtypes)

# Optimizar tipos de datos
print("\nOptimizando tipos de datos...")
start = time.time()

# Convertir columnas categóricas
df_optimized = df_vuelos.copy()
if 'Origin' in df_optimized.columns:
    df_optimized['Origin'] = df_optimized['Origin'].astype('category')
if 'Dest' in df_optimized.columns:
    df_optimized['Dest'] = df_optimized['Dest'].astype('category')

# Convertir enteros grandes a tipos más pequeños
int_columns = ['Year', 'Month', 'DayofMonth', 'DayOfWeek']
for col in int_columns:
    if col in df_optimized.columns:
        df_optimized[col] = df_optimized[col].astype('int8')

# Convertir delays a float32 para ahorrar memoria
float_columns = ['DepDelay', 'ArrDelay']
for col in float_columns:
    if col in df_optimized.columns:
        df_optimized[col] = df_optimized[col].astype('float32')

optimization_time = time.time() - start
print(f"Optimización completada en {optimization_time:.3f} segundos")

print("\nTipos de datos optimizados:")
print(df_optimized.dtypes)

## 7. DEMOSTRACIÓN PRÁCTICA: DATASET DE 1GB

### Objetivo: Procesar un dataset realista de ~1GB

Vamos a:
1. **Generar** un dataset de 1GB de datos de e-commerce
2. **Analizar** patrones de comportamiento de usuarios
3. **Optimizar** queries usando las mejores prácticas de Dask

In [ ]:
# Función para generar dataset grande - VERSIÓN CORREGIDA
def generar_dataset_ecommerce(tamaño_gb=0.5):
    """
    Genera un dataset de e-commerce del tamaño especificado - VERSIÓN SIN ERRORES
    Estimación: ~100MB por millón de filas
    """
    print(f"🚀 Generando dataset de {tamaño_gb}GB...")
    
    # Calcular número de filas basado en tamaño deseado
    filas_por_mb = 10000  # Estimación
    total_filas = int(tamaño_gb * 1000 * filas_por_mb)  # 100MB por millón de filas
    
    print(f"Generando {total_filas:,} filas...")
    
    # Crear datos sintéticos realistas
    np.random.seed(42)  # Para reproducibilidad
    
    start_time = time.time()
    
    # Generar en chunks para memoria eficiente
    chunk_size = 100000
    chunks = []
    
    productos = ['Laptop', 'Smartphone', 'Tablet', 'Auriculares', 'Monitor', 
                'Teclado', 'Mouse', 'Impresora', 'Webcam', 'Micrófono',
                'Smartwatch', 'Cámara', 'Parlantes', 'Disco Duro', 'RAM']
    
    categorias = ['Electrónicos', 'Accesorios', 'Computadoras', 'Audio', 'Video']
    
    usuarios = [f'usuario_{i}' for i in range(1, 100001)]  # 100k usuarios únicos
    
    for i in range(0, total_filas, chunk_size):
        current_size = min(chunk_size, total_filas - i)
        
        # ✅ CORRECCIÓN: Generar fechas directamente sin shift()
        # En lugar de usar shift() que causaba problemas con arrays
        fecha_base = pd.Timestamp('2023-01-01')
        # Generar offsets aleatorios en minutos
        offsets_minutos = np.random.randint(-365*24*60, 0, current_size)
        fechas_compra = pd.to_datetime(fecha_base + pd.to_timedelta(offsets_minutos, unit='m'))
        
        chunk = pd.DataFrame({
            'user_id': np.random.choice(usuarios, current_size),
            'producto': np.random.choice(productos, current_size),
            'categoria': np.random.choice(categorias, current_size),
            'precio': np.random.uniform(20, 2000, current_size).round(2),
            'cantidad': np.random.poisson(2, current_size) + 1,  # Poisson para naturalidad
            'descuento': np.random.uniform(0, 0.3, current_size).round(2),  # 0-30% descuento
            'fecha_compra': fechas_compra,  # ✅ SOLUCIÓN: Fechas generadas directamente
            'pais': np.random.choice(['España', 'Francia', 'Alemania', 'Italia', 'Portugal'], current_size),
            'metodo_pago': np.random.choice(['Tarjeta', 'PayPal', 'Transferencia', 'Bizum'], current_size),
            'rating': np.random.randint(1, 6, current_size)  # 1-5 estrellas
        })
        
        # Calcular precio final con descuento
        chunk['precio_final'] = (chunk['precio'] * (1 - chunk['descuento']) * chunk['cantidad']).round(2)
        
        chunks.append(chunk)
        
        if i % 1000000 == 0 and i > 0:
            print(f"  Generadas {i:,} filas...")
    
    # Combinar chunks
    df_completo = pd.concat(chunks, ignore_index=True)
    
    # Calcular tamaño real
    tamaño_mb = df_completo.memory_usage(deep=True).sum() / 1024 / 1024
    
    print(f"✅ Dataset generado en {time.time() - start_time:.2f} segundos")
    print(f"📊 Filas: {len(df_completo):,}")
    print(f"💾 Tamaño en memoria: {tamaño_mb:.1f} MB")
    
    return df_completo

print("✅ Función corregida lista - Ya no hay errores de array ambiguity")

In [ ]:
# Generar dataset (AHORA SIN ERRORES)
print("🔥 GENERANDO DATASET...")
df_ecommerce = generar_dataset_ecommerce(tamaño_gb=0.5)  # 500MB por tiempo del curso
print(f"\nDataset listo para análisis: {df_ecommerce.shape}")
print(f"📋 Columnas: {list(df_ecommerce.columns)}")
print(f"\n🔍 Primeras 3 filas:")
df_ecommerce.head(3)

In [ ]:
# Convertir a DataFrame de Dask
print("🔄 Convirtiendo a DataFrame de Dask...")
start_time = time.time()

# Dividir en particiones basadas en el número de CPUs
import os
n_cores = os.cpu_count() or 4
n_partitions = min(n_cores * 2, 8)  # Doble número de cores, máximo 8

print(f"Usando {n_partitions} particiones en {n_cores} cores")

df_dask_ecommerce = dd.from_pandas(df_ecommerce, npartitions=n_partitions)

print(f"✅ Conversión completada en {time.time() - start_time:.2f} segundos")
print(f"Forma: {df_dask_ecommerce.shape}")
print(f"Particiones: {df_dask_ecommerce.npartitions}")

### Análisis 1: Performance Comparison

**Compararemos la velocidad entre Pandas y Dask:**

In [ ]:
# Comparación de performance: Pandas vs Dask
print("⚡ COMPARACIÓN DE PERFORMANCE")
print("=" * 50)

# Operación 1: Filtrado simple
print("\n1. Filtrado de compras > $100:")

# Pandas
start = time.time()
pandas_result = df_ecommerce[df_ecommerce['precio_final'] > 100]
pandas_time = time.time() - start
print(f"   Pandas: {pandas_time:.2f} segundos - {len(pandas_result):,} resultados")

# Dask
start = time.time()
dask_result = df_dask_ecommerce[df_dask_ecommerce['precio_final'] > 100].compute()
dask_time = time.time() - start
print(f"   Dask:   {dask_time:.2f} segundos - {len(dask_result):,} resultados")

print(f"   🚀 Dask es {pandas_time/dask_time:.1f}x más rápido" if dask_time < pandas_time else f"   🐌 Pandas es {dask_time/pandas_time:.1f}x más rápido")

# Operación 2: GroupBy complejo
print("\n2. Ventas por categoría y país:")

# Pandas
start = time.time()
pandas_groupby = df_ecommerce.groupby(['categoria', 'pais'])['precio_final'].agg(['sum', 'mean', 'count'])
pandas_time = time.time() - start
print(f"   Pandas: {pandas_time:.2f} segundos")

# Dask
start = time.time()
dask_groupby = df_dask_ecommerce.groupby(['categoria', 'pais'])['precio_final'].agg(['sum', 'mean', 'count']).compute()
dask_time = time.time() - start
print(f"   Dask:   {dask_time:.2f} segundos")

print(f"   🚀 Dask es {pandas_time/dask_time:.1f}x más rápido" if dask_time < pandas_time else f"   🐌 Pandas es {dask_time/pandas_time:.1f}x más rápido")

### Análisis 2: Insights de Negocio

**Vamos a extraer insights útiles del dataset:**

In [ ]:
# Análisis de negocio con Dask
print("📈 ANÁLISIS DE NEGOCIO")
print("=" * 50)

# 1. Top productos por ingresos
print("\n🏆 Top 5 productos por ingresos totales:")
start = time.time()
top_productos = df_dask_ecommerce.groupby('producto')['precio_final'].sum().nlargest(5).compute()
print(f"Tiempo: {time.time() - start:.2f} segundos")
for i, (producto, ingresos) in enumerate(top_productos.items(), 1):
    print(f"   {i}. {producto}: ${ingresos:,.2f}")

# 2. Análisis temporal
print("\n📅 Ingresos por mes:")
start = time.time()
df_dask_ecommerce['mes'] = df_dask_ecommerce['fecha_compra'].dt.to_period('M')
ingresos_mes = df_dask_ecommerce.groupby('mes')['precio_final'].sum().compute()
print(f"Tiempo: {time.time() - start:.2f} segundos")
print("Primeros 5 meses:")
print(ingresos_mes.head())

# 3. Segmentación de clientes
print("\n👥 Segmentación por gasto total:")
start = time.time()
gasto_usuarios = df_dask_ecommerce.groupby('user_id')['precio_final'].sum().compute()
gasto_usuarios = gasto_usuarios.reset_index()
gasto_usuarios['segmento'] = pd.cut(gasto_usuarios['precio_final'], 
                                   bins=[0, 100, 500, 1000, float('inf')], 
                                   labels=['Bajo', 'Medio', 'Alto', 'VIP'])
segmentos = gasto_usuarios['segmento'].value_counts()
print(f"Tiempo: {time.time() - start:.2f} segundos")
print("Distribución de segmentos:")
for segmento, cantidad in segmentos.items():
    porcentaje = (cantidad / len(gasto_usuarios)) * 100
    print(f"   {segmento}: {cantidad:,} usuarios ({porcentaje:.1f}%)")

## 8. EJERCICIO FINAL: TU TURNO

### **Tiempo: 10 minutos**

**Objetivo**: Analizar patrones de comportamiento de usuarios usando Dask

**Dataset**: Usa el mismo `df_dask_ecommerce` que hemos estado trabajando

**Tareas**:
1. **Identifica** el top 3 de usuarios más rentables
2. **Calcula** el valor promedio de transacción por método de pago
3. **Encuentra** la correlación entre rating y precio final
4. **Determina** qué día de la semana genera más ventas

**Instrucciones**:
- Escribe el código completo en la celda siguiente
- Usa `.compute()` solo cuando sea necesario
- Muestra los resultados de forma clara
- Explica qué insights obtienes

In [27]:
# 📝 EJERCICIO FINAL - Completa el código

print("🎯 EJERCICIO FINAL: Análisis de Comportamiento de Usuarios")
print("=" * 60)

# Tu código aquí:

# 1. TOP 3 USUARIOS MÁS RENTABLES
print("\n1. 🏆 TOP 3 USUARIOS MÁS RENTABLES:")
# Hint: usa groupby('user_id')['precio_final'].sum() y .nlargest(3)




# 2. VALOR PROMEDIO POR MÉTODO DE PAGO
print("\n2. 💳 VALOR PROMEDIO POR MÉTODO DE PAGO:")
# Hint: usa groupby('metodo_pago')['precio_final'].mean()




# 3. CORRELACIÓN ENTRE RATING Y PRECIO
print("\n3. ⭐ CORRELACIÓN RATING-PRECIO:")
# Hint: usa .corr() entre las columnas 'rating' y 'precio_final'




# 4. DÍA DE LA SEMANA CON MÁS VENTAS
print("\n4. 📅 MEJOR DÍA DE LA SEMANA:")
# Hint: extrae day_name() de fecha_compra y agrupa por ese campo




# 💡 INSIGHTS ADICIONALES
print("\n5. 💡 TUS INSIGHTS:")
# ¿Qué patrones interesante encuentras?
# ¿Qué recomendaciones harías al negocio?




print("\n✅ ¡Ejercicio completado!")

🎯 EJERCICIO FINAL: Análisis de Comportamiento de Usuarios

1. 🏆 TOP 3 USUARIOS MÁS RENTABLES:

2. 💳 VALOR PROMEDIO POR MÉTODO DE PAGO:

3. ⭐ CORRELACIÓN RATING-PRECIO:

4. 📅 MEJOR DÍA DE LA SEMANA:

5. 💡 TUS INSIGHTS:

✅ ¡Ejercicio completado!


## 9. RESUMEN Y CASOS DE USO

### Lo que has aprendido en este curso:

1. **Lazy Evaluation**: Dask construye planes de ejecución, no ejecuta inmediatamente
2. **Particiones**: Cómo Dask divide los datos para paralelización eficiente
3. **API Familiar**: Sintaxis similar a Pandas pero distribuida y escalable
4. **Performance**: Cuándo usar Dask vs Pandas según el tamaño del dataset
5. **Optimizaciones**: Tipos de datos, particionamiento y limpieza de datos
6. **Casos Prácticos**: Análisis real de vuelos con datasets grandes

### Casos de uso ideales para Dask:

- Datasets de 1GB+ en laptop personal
- Análisis de logs y datos de sensores
- Procesamiento ETL distribuido
- Machine Learning con datasets grandes
- Análisis financiero en tiempo real
- Procesamiento de datos científicos

### Cuándo usar Dask vs Pandas:

| Situación | Recomendación |
|-----------|---------------|
| Dataset < 100MB | Pandas (más rápido) |
| Dataset 100MB - 1GB | Dask (si hay múltiples núcleos) |
| Dataset > 1GB | Dask (necesario) |
| Datasets que no caben en RAM | Dask (lazy loading) |
| Análisis simple de una vez | Pandas |
| Pipeline complejo distribuido | Dask |


## 8. RECURSOS ADICIONALES

### Documentación y Aprendizaje

- Documentación oficial: https://docs.dask.org/en/stable/
- Tutorial interactivo: https://tutorial.dask.org
- Comunidad: https://stackoverflow.com/questions/tagged/dask
- Ejemplos reales: https://github.com/dask/dask-examples
- Guía de migración de Pandas: https://docs.dask.org/en/stable/dataframe-api.html

### Próximos Pasos

1. **Practica** con tus propios datasets
2. **Experimenta** con diferentes números de particiones
3. **Aprende** Dask-ML para machine learning
4. **Explora** Dask-Delayed para funciones personalizadas
5. **Considera** clusters distribuidos para datasets masivos